# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [34]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import folium
import random

import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [29]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lihue,21.9789,-159.3672,22.08,89,100,5.66,US,1706301761
1,1,adamstown,-25.0660,-130.1015,24.48,73,100,3.83,PN,1706301761
2,2,bethel,41.3712,-73.4140,6.20,86,100,2.06,US,1706301761
3,3,talnakh,69.4865,88.3972,-13.77,93,100,7.27,RU,1706301762
4,4,port mathurin,-19.6833,63.4167,27.55,84,100,5.52,MU,1706301762


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [69]:
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.75,
    color = "City"
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [58]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df['Max Temp']<=30)\
                                     &(city_data_df['Max Temp']>=10)\
                                     &(city_data_df['Cloudiness']<=0)\
                                     &(city_data_df['Humidity']<60),:]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
57,57,ushuaia,-54.8000,-68.3000,18.81,42,0,3.09,AR,1706301770
62,62,punta arenas,-53.1500,-70.9167,22.01,32,0,5.14,CL,1706301771
63,63,bardai,11.6500,7.2333,18.83,23,0,6.28,NG,1706301771
71,71,pericos,25.0500,-107.7000,27.95,16,0,3.21,MX,1706301772
72,72,i-n-salah,27.1935,2.4607,17.77,22,0,7.88,DZ,1706301773


### Step 3: Create a new DataFrame called `hotel_df`.

In [59]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
57,ushuaia,AR,-54.8000,-68.3000,42,
62,punta arenas,CL,-53.1500,-70.9167,32,
63,bardai,NG,11.6500,7.2333,23,
71,pericos,MX,25.0500,-107.7000,16,
72,i-n-salah,DZ,27.1935,2.4607,22,
101,la mira,MX,18.0350,-102.3266,55,
104,calama,CL,-22.4667,-68.9333,24,
127,san luis de la loma,MX,17.2706,-100.8939,58,
167,kidal,ML,18.4411,1.4078,24,
200,san jose temascatio,MX,20.6833,-101.2667,12,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [60]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
punta arenas - nearest hotel: Hotel Lacolet
bardai - nearest hotel: No hotel found
pericos - nearest hotel: No hotel found
i-n-salah - nearest hotel: باجودة
la mira - nearest hotel: Hotel La Loma
calama - nearest hotel: Apart Hotel Geotel Calama
san luis de la loma - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
san jose temascatio - nearest hotel: Irapuato
rio gallegos - nearest hotel: Amérian Río Gallegos Apart & Suites
tazacorte - nearest hotel: App Leyma
tera - nearest hotel: Hôtel Daressalam Téra
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
villa gesell - nearest hotel: Hotel El Faro
tindouf - nearest hotel: محمد بوسبي
cabo san lucas - nearest hotel: Comfort Rooms
loreto - nearest hotel: No hotel found
murzuq - nearest hotel: فندق باريس - مرزق
tamanrasset - nearest hotel: فندق الأمان
ciudad lazaro cardena

,City,Country,Lat,Lng,Humidity,Hotel Name
57,ushuaia,AR,-54.8000,-68.3000,42,Apart Hotel Aires del Beagle
62,punta arenas,CL,-53.1500,-70.9167,32,Hotel Lacolet
63,bardai,NG,11.6500,7.2333,23,No hotel found
71,pericos,MX,25.0500,-107.7000,16,No hotel found
72,i-n-salah,DZ,27.1935,2.4607,22,باجودة
101,la mira,MX,18.0350,-102.3266,55,Hotel La Loma
104,calama,CL,-22.4667,-68.9333,24,Apart Hotel Geotel Calama
127,san luis de la loma,MX,17.2706,-100.8939,58,No hotel found
167,kidal,ML,18.4411,1.4078,24,No hotel found
200,san jose temascatio,MX,20.6833,-101.2667,12,Irapuato


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [65]:

hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)